# Cube dual volume computation

In this section we benchmark different methods for compute dual volumes on a simple triagulated cube (found using hyperct

In [69]:
import numpy as np
from scipy.spatial import HalfspaceIntersection
import matplotlib.pyplot as plt
import polyscope as ps


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
from ipywidgets import *
from matplotlib.widgets import Slider
from lsm import HNdC_ijk

# ddg imports
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from ddgclib._complex import Complex
from ddgclib import *
from ddgclib._complex import *
from ddgclib._curvatures import * #plot_surface#, curvature
from ddgclib._capillary_rise_flow import * #plot_surface#, curvature
from ddgclib._hyperboloid import *
from ddgclib._catenoid import *
from ddgclib._ellipsoid import *
from ddgclib._eos import *
from ddgclib._misc import *
from ddgclib._plotting import *
from ddgclib._sphere import *


def polytope_volume(cutting_planes):
    """
    Computes the volume of a convex polytope defined by its linear cutting planes.
    
    Parameters:
    cutting_planes (ndarray): An array of shape (m, d + 1), where m is the number of cutting planes and d is the dimension of the polytope. Each row of the array corresponds to a cutting plane in the form [a_1, a_2, ..., a_d, b], where a_i are the coefficients of the hyperplane equation and b is the constant term.
    
    Returns:
    float: The volume of the polytope.
    """
    # Create an instance of HalfspaceIntersection with the cutting planes
    hs = HalfspaceIntersection(cutting_planes[:,:-1], -cutting_planes[:,-1])
    
    # Compute the volume of the polytope using the HalfspaceIntersection object
    volume = hs.volume
    
    return volume

import numpy as np

def polytope_volume_gram(vertices):
    """
    Computes the volume of a convex polytope defined by its vertices using Gram's relation.
    
    Parameters:
    vertices (ndarray): An array of shape (n, d), where n is the number of vertices and d is the dimension of the polytope.
    
    Returns:
    float: The volume of the polytope.
    """
    # Calculate the Gram matrix
    gram_matrix = np.dot(vertices.T, vertices)
    
    # Compute the determinant of the Gram matrix
    det_gram_matrix = np.linalg.det(gram_matrix)
    
    # Compute the volume of the polytope using Gram's relation
    volume = np.sqrt(np.abs(det_gram_matrix)) / np.math.factorial(vertices.shape[0])
    
    return volume



In [70]:
from ddgclib._complex import *  # hyperct 

In [71]:
bounds = [(-1, 3),] * 3  # 4**2  = 16 m3
HC = Complex(3, domain=bounds)

In [72]:
HC.triangulate()
HC.refine_all()
HC.refine_all()



In [ ]:
HC.plot_complex()
plt.show()

/home/endres/projects/ddg/ddgclib/ddgclib/_complex.py:2832: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig_complex.show()


In [ ]:
plot_polyscope(HC)
ps.show()

In [52]:
HC.V.cache

Va = []
for v in HC.V:
    Va.append(v.x)
    
Va = np.array(X ,dtype='float64')
Va

array([[-1. , -1. , -1. ],
       [ 3. ,  3. ,  3. ],
       [ 3. , -1. , -1. ],
       [-1. ,  3. , -1. ],
       [ 3. ,  3. , -1. ],
       [-1. , -1. ,  3. ],
       [ 3. , -1. ,  3. ],
       [-1. ,  3. ,  3. ],
       [ 1. ,  1. ,  1. ],
       [ 1. , -1. , -1. ],
       [ 1. ,  1. , -1. ],
       [-1. ,  1. , -1. ],
       [ 3. ,  1. , -1. ],
       [ 1. ,  3. , -1. ],
       [ 1. ,  1. ,  3. ],
       [-1. , -1. ,  1. ],
       [ 3. , -1. ,  1. ],
       [-1. ,  3. ,  1. ],
       [ 3. ,  3. ,  1. ],
       [ 1. , -1. ,  1. ],
       [-1. ,  1. ,  1. ],
       [ 3. ,  1. ,  1. ],
       [ 1. ,  3. ,  1. ],
       [ 1. , -1. ,  3. ],
       [-1. ,  1. ,  3. ],
       [ 3. ,  1. ,  3. ],
       [ 1. ,  3. ,  3. ],
       [ 2. ,  0. ,  0. ],
       [ 2. ,  0. ,  2. ],
       [ 0. ,  2. ,  0. ],
       [ 0. ,  0. ,  0. ],
       [ 2. ,  2. ,  2. ],
       [ 0. ,  2. ,  2. ],
       [ 0. ,  0. ,  2. ],
       [ 2. ,  2. ,  0. ],
       [ 2. , -1. , -1. ],
       [ 2. ,  0. , -1. ],
 

In [57]:

X

array([[ 0. , -1. , -1. ],
       [ 0. , -1. ,  1. ],
       [ 0. ,  1. ,  0. ],
       [ 0.5, -0.5,  0.5],
       [ 0.5, -0.5, -0.5],
       [ 0. ,  0. , -1. ],
       [ 0. ,  0. ,  1. ],
       [ 1. ,  0. ,  1. ],
       [ 1. ,  1. ,  0. ],
       [ 1. ,  0. , -1. ],
       [ 0. , -1. ,  0. ],
       [-1. ,  1. ,  0. ],
       [ 1. , -1. ,  0. ],
       [ 0.5,  0.5, -0.5],
       [ 0.5,  0.5,  0.5],
       [-0.5, -0.5, -0.5],
       [-0.5, -0.5,  0.5],
       [-1. ,  0. ,  1. ],
       [-1. ,  0. , -1. ],
       [ 1. ,  0. ,  0. ],
       [-1. , -1. ,  0. ],
       [ 0. ,  1. , -1. ],
       [ 0. ,  1. ,  1. ],
       [-1. ,  0. ,  0. ],
       [-0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5]], dtype=float128)

In [66]:
# Construct cutting_planes

# half spaces are Stacked Inequalities of the form Ax + b <= 0 in format [A; b]
# A = np.hstack((halfspaces[:, :-1], norm_vector))
#b = - halfspaces[:, -1:]


# Contants of hyperplanes are half the norm of the vector:
i = 30
HC.V[tuple(Va[i])]
vi = HC.V[tuple(Va[i])]
Xi = vi.x_a
X = []
for v in vi.nn:
    X.append(v.x)
    
X = np.array(X)

A = X - Xi
A_norm = normalized(A)
b = np.sum(0.5 * A * A_norm, axis=1)  # For some reason no minus factor like formula
A = A_norm

# NOTE: Remove zero vectors
A = A[mask]
b = b[mask]

In [67]:
cutting_planes = np.column_stack((A, -b))
cutting_planes

array([[ 0.        , -0.70710678, -0.70710678, -0.70710678],
       [ 0.        , -0.70710678,  0.70710678, -0.70710678],
       [ 0.        ,  1.        ,  0.        , -0.5       ],
       [ 0.57735027, -0.57735027,  0.57735027, -0.4330127 ],
       [ 0.57735027, -0.57735027, -0.57735027, -0.4330127 ],
       [ 0.        ,  0.        , -1.        , -0.5       ],
       [ 0.        ,  0.        ,  1.        , -0.5       ],
       [ 0.70710678,  0.        ,  0.70710678, -0.70710678],
       [ 0.70710678,  0.70710678,  0.        , -0.70710678],
       [ 0.70710678,  0.        , -0.70710678, -0.70710678],
       [ 0.        , -1.        ,  0.        , -0.5       ],
       [-0.70710678,  0.70710678,  0.        , -0.70710678],
       [ 0.70710678, -0.70710678,  0.        , -0.70710678],
       [ 0.57735027,  0.57735027, -0.57735027, -0.4330127 ],
       [ 0.57735027,  0.57735027,  0.57735027, -0.4330127 ],
       [-0.57735027, -0.57735027, -0.57735027, -0.4330127 ],
       [-0.57735027, -0.

In [68]:
from scipy.spatial import HalfspaceIntersection

#Xi  # Current interior point
halfspaces = cutting_planes
hs = HalfspaceIntersection(halfspaces, Xi)
hs.dual_volume

28.444444444444436

In [ ]:
# # Compute the volume of the polytope using the HalfspaceIntersection object
 #   volume = hs.volume

In [44]:
X[i] 

array([0., 0., 0.])